In [80]:
import dal, pandas, common
dal.init_db()
common.PERIODS_IN_HOUR = 4*60
common.PERIODS_IN_DAY = common.PERIODS_IN_HOUR * 24
from scipy.stats import multivariate_normal
from sklearn import model_selection
import seaborn as sns
from datetime import datetime
from collections import defaultdict

import matplotlib.pyplot as plt

In [90]:
EPSILON = 2.09003339968e-11

In [2]:
%matplotlib inline

In [3]:
all_sessions = dal.get_ended_sessions()

In [128]:
all_sessios = filter(lambda x: x['src_ip'].startswith('10.0'),all_sessions)

In [139]:
filter(lambda x: same_session(all_sessions[0], x) and  all_sessions[0]['timestamp'] < x['timestamp'] < all_sessions[0]['timestamp'] -60, all_sessions)

[]

In [138]:
def ep_to_hour(ep):
    return int(datetime.fromtimestamp(ep).strftime("%H"))

def create_model(data):
    return multivariate_normal(mean = data[['sessions_bandwidths', 'sessions_durations']].mean(), cov=data[['sessions_bandwidths', 'sessions_durations']].cov())

def calc_prob(row, model):
    return  model.pdf((row['sessions_durations'], row['sessions_bandwidths']))


In [ ]:
def has_1_session(dest_ip, timestamp, all_session)

In [142]:
def get_hours_sessions(all_sessions):
    kpis_by_hours = defaultdict(list)
    
    for s in all_sessions:
        s_hour = ep_to_hour(s['timestamp'])
        kpis_by_hours[s_hour].append(s)
        
    return kpis_by_hours

df = get_hours_sessions(all_sessions)

In [69]:
def my_log(x):
    if isinstance(x, (int, long, float, complex)):
        return common.safe_log(common.safe_log(x))
    else:
        return x

In [143]:
for i in df:
    df[i] =  pandas.DataFrame(df[i])

In [144]:
for i in df:
    data = pandas.DataFrame()
    data['sessions_bandwidths'] = df[i]['n_bytes']
    data['sessions_durations'] = df[i]['timestamp'] - df[i]['start_time']
    data['dest_ip'] = df[i]['dest_ip']
    data['src_ip'] = df[i]['src_ip']
    data['src_port'] = df[i]['src_port']
    data['dest_port'] = df[i]['dest_port']
    data['timestamp'] = df[i]['timestamp']
    df[i] = data

In [145]:
for i in df:
    df[i]['sessions_bandwidths'] = df[i]['sessions_bandwidths'].apply(lambda x: my_log(x))
    df[i]['sessions_durations'] = df[i]['sessions_durations'].apply(lambda x: my_log(x))


In [146]:
models = {}
for i in df:
    models[i] =  create_model(df[i])

In [147]:
for i in df:
    df[i]['prob'] = df[i].apply(calc_prob, args=(models[i],), axis=1)

In [151]:
# drop lower prob
for i in df:
    df[i] = df[i].drop(df[i][df[i]['prob']<EPSILON].index)

In [154]:
# drop lower than average
for  i in df:
    df[i] = df[i].drop(df[i][df[i]['sessions_bandwidths']<models[i].mean[1]].index)
    df[i] = df[i].drop(df[i][df[i]['sessions_durations']<models[i].mean[0]].index)

In [ ]:
def calc_n_sessions(row):
    same_ip = filter(lambda s: s['dest_ip'] == row['dest_ip'],all_sessions)
    same_time = filter(lambda s: row['timestamp'] <s['timestamp']<row['timestamp'] - 60,same_ip)
    return len(same_time)

In [178]:
for i in df:
    df[i]['n_sessions'] = df[i].apply(calc_n_sessions, axis=1)

In [182]:
keep = df.copy()

In [209]:
d=df[0].iloc[0]['dest_ip']
filter(lambda s: s['dest_ip'] == d, all_sessions)
# df[0].iloc[0]['dest_ip']

[{u'dest_ip': u'64.233.166.189',
  u'dest_port': 443,
  u'n_bytes': 3346,
  u'protocol': 6,
  u'src_ip': u'10.0.0.100',
  u'src_port': 22557,
  u'start_time': 1495434679.516866,
  u'timestamp': 1495435820.99164},
 {u'dest_ip': u'64.233.166.189',
  u'dest_port': 443,
  u'n_bytes': 9253,
  u'protocol': 6,
  u'src_ip': u'10.0.0.100',
  u'src_port': 22557,
  u'start_time': 1495432213.19371,
  u'timestamp': 1495434600.087074},
 {u'dest_ip': u'64.233.166.189',
  u'dest_port': 443,
  u'n_bytes': 18439,
  u'protocol': 6,
  u'src_ip': u'10.0.0.17',
  u'src_port': 57296,
  u'start_time': 1495400039.941258,
  u'timestamp': 1495403714.323156},
 {u'dest_ip': u'64.233.166.189',
  u'dest_port': 443,
  u'n_bytes': 18282,
  u'protocol': 6,
  u'src_ip': u'10.0.0.17',
  u'src_port': 57038,
  u'start_time': 1495381635.066273,
  u'timestamp': 1495385296.627095},
 {u'dest_ip': u'64.233.166.189',
  u'dest_port': 443,
  u'n_bytes': 7974,
  u'protocol': 6,
  u'src_ip': u'10.0.0.100',
  u'src_port': 1455,
  u's